In [15]:
import asyncio
import os
import numpy as np
import logging
from pathlib import Path

import capnp
import typer
import zmq.asyncio
import time
from deepdrrzmq.utils.zmq_util import zmq_no_linger_context

from deepdrrzmq.utils.typer_util import unwrap_typer_param
from deepdrrzmq.utils.server_util import make_response, DeepDRRServerException, messages
import random
import string

import threading
from IPython.display import display
import ipywidgets as widgets
import time
from threading import Event
import asyncio
rep_port=40100
pub_port=40101
sub_port=40102
context = zmq.Context()
context.__enter__()

sub_socket = context.socket(zmq.SUB)
sub_socket.hwm = 10000

pub_socket = context.socket(zmq.PUB)
pub_socket.hwm = 10000

addr = "localhost"
pub_socket.connect(f"tcp://{addr}:{pub_port}") # make variable
sub_socket.connect(f"tcp://{addr}:{sub_port}")


sub_socket.subscribe(b"/loggerd/")

In [16]:
record_button = widgets.Button(description='New Session', disabled=True, button_style='', icon='circle')
stop_button = widgets.Button(description='Stop', disabled=True, button_style='', icon='square')

session_id = widgets.Text(value='None', placeholder='', description='Session ID:', disabled=True)
record_status = widgets.Text(value='None', placeholder='', description='Recording:', disabled=True)

async def f():
    last_heard_time = 0

    while True:
        latest_msgs = {}

        try:
            for i in range(1000):
                topic, data = sub_socket.recv_multipart(flags=zmq.NOBLOCK)
                latest_msgs[topic] = data
        except zmq.ZMQError:
            pass

        for topic, data in latest_msgs.items():
            if topic == b"/loggerd/status/":
                last_heard_time = time.time()
                record_button.disabled = False
                stop_button.disabled = False
                with messages.LoggerStatus.from_bytes(data) as msg:
                    session_id.value = msg.sessionId if msg.recording else "no session"
                    record_status.value = str(msg.recording)
                    record_button.button_style = 'danger' if msg.recording else ''

        if time.time() - last_heard_time > 3:
            session_id.value = "Disconnected"
            record_status.value = "Disconnected"
            record_button.disabled = True
            stop_button.disabled = True

        await asyncio.sleep(0.1)

try:
    ui_update_task.cancel()
except:
    pass
ui_update_task = asyncio.create_task(f())

def on_record_button_clicked(b):
    pub_socket.send_multipart([b"/loggerd/start/", b""])
    

def on_stop_button_clicked(b):
    pub_socket.send_multipart([b"/loggerd/stop/", b""])

colortoggle = widgets.ToggleButtons(
    options=[('Opaque', True), ('Transparent' ,False)],
    description='Patient Material:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

updatesetting = widgets.Button(
    description='Update Setting',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)


left_box = widgets.VBox([colortoggle])
display(widgets.HBox([left_box,updatesetting]))

def opdatesetting_clicked(b):
    response_topic = "/mp/setting/changeskinopaque/--OBJECT--/DRRWorldOrigin/PatientOrigin/Userstudy/teleport_and_changematerial/"
    
    # response_topic = "/mp/setting/webgui/SettingManager/"
    msg = messages.SycnedSetting.new_message()
    msg.timestamp = msg.timestamp = float(np.float64(time.time()).item())
    msg.clientId = 'webgui'
    uiControlVar = msg.setting.init("skinandtransport")
    uiControlVar.skinopaqueweb = colortoggle.value
    pub_socket.send_multipart([response_topic.encode(), msg.to_bytes()])
    print(colortoggle.value)

updatesetting.on_click(opdatesetting_clicked)
record_button.on_click(on_record_button_clicked)
record_button.on_click(opdatesetting_clicked)
stop_button.on_click(on_stop_button_clicked)
display(widgets.HBox([record_button, stop_button, session_id, record_status]))


False
True
False
False
